In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd

In [2]:
#Making a function to extract data from a .out file, and put it in a dataframe
#This requires the output file to be correctly formatted(18 lines per entry)
def ConvertOutputToDataframe(filepath, objectname, verbose=False):
    '''
    This function takes an output file from cubechi2, and formats the relevant information
    into a pandas dataframe. This needs to have the same formatting as cubechi2 outputs for it to work
    ---------------------------------------------------------------
    Input
    filepath: A complete filepath string, including filename
    objectname: A string naming the source, useful for me so I don't forget to change it here lol
    
    Output
    Table: A pandas dataframe with the columns mentioned below. All data is extracted from cubechi2 output file
    Filename, Mass, Rotation Direction, Rin, Rout, Rcb, Normalization Constant, Chi2, Reduced Chi2, Threshold Chi2, Threshold Reduced Chi2
    '''
    Cubename=[]
    MassList=[]
    RotationList=[]
    RinList=[]
    RoutList=[]
    RcbList=[]
    NormCList=[]
    AllChi2List=[]
    AllReducedChi2List=[]
    ThresholdChi2List=[]
    ThresholdReducedChi2List=[]
    
    ThresholdNumCellsList=[]
    ThresholdNumCellsOverBeamAreaList=[]
    


    #Put complete filepath here
    #'/Volumes/REU_2023/Per-emb-14_Models/KeplerianOnly/ModelsJune23/KeplerianJune23.out'
    
    file=open(filepath) #Opening the output file
    filelist=file.readlines()
    
    
    #print(filelist[0:19])
    temp=filelist[7].find('Beam') #First extracting beam information, since it is just a 1 time thing
    BMaj=float(filelist[7][temp+4:temp+9])
    BMin=float(filelist[7][temp+11:temp+15])
    temp=filelist[7].find('Pix')
    Pixsize=float(filelist[7][temp+3:temp+8])
    BeamAreaPix=(BMaj * BMin * np.pi)/(Pixsize**2)
    
    if verbose==True:
        print('BMin=',BMin)
        print('BMaj=',BMaj)
        print('Pixsize=',Pixsize)
        print('BeamAreaPix=',BeamAreaPix) #Testing to make sure I implimented this correct lol
    
    
    
    if verbose==True:
        print(filepath, 'sucessfully read')
        print(((len(filelist)+1)/19), ' models were compared')
        
    for i in range(int((len(filelist)+1)/19)): #Main loop, gpes through output file in groups of 19 lines and collects the data
    
        tempname=filelist[i*19+7] #Starts with model cube name
        HelpfulReference=tempname.find('/'+objectname+'-Vsys')+len(objectname) #Just a nice index to start at the actual filename
        j=0
    
        Cubename_Start=tempname.find(objectname + '-Vsys')
        Cubename.append(tempname[Cubename_Start:])#Grabs the main name of the cube
        Cubename[i]=Cubename[i][:-1] #Remove newline
    
        MassStart=tempname.find('M', HelpfulReference) #Find where the mass is in the filename
        MassList.append(tempname[MassStart+1:MassStart+4]) #And pull it out to store
    
        RotationStart=tempname.find('Rot', HelpfulReference)
        if tempname[RotationStart + 3]=='-':
            RotationList.append(tempname[RotationStart + 3:RotationStart + 5])
        else:
            RotationList.append(tempname[RotationStart + 3])
    
        Rin_Start=tempname.find('Rin', HelpfulReference) #find Rin in filename
        while tempname[Rin_Start + 3 + j].isdigit() == True : #little loop to handle different digit length numbers, simply runs until the next chracter is not a digit
            j+=1 #And keeps track of how long it has searched
        RinList.append(tempname[Rin_Start +3:Rin_Start +3 + j])
        j=0
    
        Rout_Start=tempname.find('Rout', HelpfulReference) #find Rout in the filename
        while tempname[Rout_Start + 4 + j].isdigit() == True:
            j+=1
        RoutList.append(tempname[Rout_Start + 4:Rout_Start + 4 + j])
        j=0
    
        Rcb_Start=tempname.find('CB', HelpfulReference) 
        while tempname[Rcb_Start +2 + j].isdigit() == True:
            j+=1
        RcbList.append(tempname[Rcb_Start+2:Rcb_Start + 2 + j])
        j=0

        #Finished with cube filename stuff, now onto outputs
    
        NormCList.append(float(filelist[i*19+9][-12:]))
        AllChi2List.append(float(filelist[i*19+12][-12:]))
        AllReducedChi2List.append(float(filelist[i*19+13][-12:]))
        ThresholdChi2List.append(float(filelist[i*19+16][-12:]))
        ThresholdReducedChi2List.append(float(filelist[i*19+17][-12:]))
        ThresholdNumCellsList.append(float(filelist[i*19+15][36:]))
        ThresholdNumCellsOverBeamAreaList.append(float(ThresholdNumCellsList[i] / BeamAreaPix))
      
    if verbose==True:
        print('File sucessfully extracted')
        
    #Now make it a dataframe
    TableTemp=pd.DataFrame([Cubename, MassList, RotationList, RinList, RoutList, RcbList, NormCList, AllChi2List, AllReducedChi2List, ThresholdChi2List, ThresholdReducedChi2List, ThresholdNumCellsList, ThresholdNumCellsOverBeamAreaList])
    Table=TableTemp.T
    Table.columns = ['Cube Name','Mass', 'RotationDirection', 'Rin', 'Rout', 'CB', 'Normalization Constant', 'Chi2', 'Reduced Chi2', 'Threshold Chi2','Threshold Reduced Chi2', 'ThresholdNumCells', 'ThresholdNumCellsOverBeamArea']
    Table.insert(6, 'IRE?', Table['CB']==Table['Rin'])
    Table.insert(7, 'Keplerian?', Table['CB']==Table['Rout'])
    Table.insert(8, 'Blended?', ~(Table['IRE?'] | Table['Keplerian?']))
    pd.set_option('display.max_columns', 10000) #replace n with the number of columns you want to see completely
    pd.set_option('display.max_rows', 10000)
    #display(Table)
    
    if verbose==True:
        print('File sucessfully converted to dataframe')
        
    return(Table)

In [3]:
def ConvertDataframeToCSV(Dataframe, filepath, verbose=False):
    '''
    This simple function will take a pandas Dataframe, and write it out as a csv.
    --------------------------------------------
    Inputs
    Dataframe: The panda Dataframe you wish to export
    filepath: A complete filepath string, including filename
    
    Outputs
    Will not return anything in python, will make a csv
    '''
    
    Dataframe.to_csv(filepath)
    
    if verbose==True:
        print('Dataframe has been exported to ', filepath)

In [4]:
def Merge3Dataframes(Table1, Table2, Table3, verbose=False):
    '''
    This function will merge 3 dataframes, and reindex for the one table
    ------------------------------------------------
    Inputs
    Table1-3: 3x pandas Dataframes you wish to merge
    
    Output
    A single reindexed Dataframe
    '''
    
    if verbose==True:
        print('Tables will be merged')
        
    BigTable=pd.concat([Table1, Table2, Table3])
    BigTable=BigTable.reset_index(drop=True)
    return(BigTable)

In [6]:
#This will do all the work of getting the output files to 1 csv
#Place all filepaths below
Output1='/Volumes/REU_2023/Per-emb-8_Models/Extension_Work/IREAndKeplerian/Per8ExtensionBlended.out'
Output2='/Volumes/REU_2023/Per-emb-8_Models/Extension_Work/IREOnly/Per8ExtensionIRE.out'
Output3='/Volumes/REU_2023/Per-emb-8_Models/Extension_Work/KeplerianOnly/Per8ExtensionKEP.out'
CSVFilepath='/Volumes/REU_2023/Per-emb-8_Models/Per8Extension.csv'
ObjectName='Per-emb-8'

#Magic happens below
TableOne=ConvertOutputToDataframe(Output1, ObjectName, verbose=True)
TableTwo=ConvertOutputToDataframe(Output2, ObjectName, verbose=True) #Extract data from out file to a dataframe
TableThree=ConvertOutputToDataframe(Output3, ObjectName, verbose=True)

BigTable=Merge3Dataframes(TableOne, TableTwo, TableThree, verbose=True) #Merge dataframes into 1 

ConvertDatafromeToCSV(BigTable, CSVFilepath, verbose=True) #And out it goes

BMin= 0.395
BMaj= 0.556
Pixsize= 0.054
BeamAreaPix= 236.6106236561696
/Volumes/REU_2023/Per-emb-8_Models/Extension_Work/IREAndKeplerian/Per8ExtensionBlended.out sucessfully read
5520  models were compared
File sucessfully extracted
File sucessfully converted to dataframe
BMin= 0.395
BMaj= 0.556
Pixsize= 0.054
BeamAreaPix= 236.6106236561696
/Volumes/REU_2023/Per-emb-8_Models/Extension_Work/IREOnly/Per8ExtensionIRE.out sucessfully read
1020  models were compared
File sucessfully extracted
File sucessfully converted to dataframe
BMin= 0.395
BMaj= 0.556
Pixsize= 0.054
BeamAreaPix= 236.6106236561696
/Volumes/REU_2023/Per-emb-8_Models/Extension_Work/KeplerianOnly/Per8ExtensionKEP.out sucessfully read
1020  models were compared
File sucessfully extracted
File sucessfully converted to dataframe
Tables will be merged
Dataframe has been exported to  /Volumes/REU_2023/Per-emb-8_Models/Per8Extension.csv


In [5]:
#Importing data from a CSV
#Right click on file, hold 'option', copy as Pathname ez pz
def CSVToDataframe(filepath, verbose=False):
    '''
    Just a really simple function to read in a csv
    '''
    Table=pd.read_csv(filepath)
    if verbose == True:
        print(filepath, 'read in')
    return(Table)

#The following function I used to combine multiple batches of model comparison, if/when we updated our search parameters
def Merge2CSVs(CSV1, CSV2, filepath):
    '''
    Takes 2x CSV filepaths, and merges into a new CSV filepath
    
    Inputs:
    CSV1, CSV2- Filepaths to CSV files you wish to combine
    filepath- filepath of new CSV location
    '''
    Table1=CSVToDataframe(CSV1)
    Table2=CSVToDataframe(CSV2)
    BigTable=pd.concat([Table1, Table2])
    BigTable=BigTable.reset_index(drop=True)
    ConvertDatafromeToCSV(BigTable, filepath)
    

In [10]:
Merge2CSVs(
           '/Volumes/REU_2023/Per-emb-8_Models/Per8Extension.csv',
           '/Volumes/REU_2023/MasterCSVs/Per-emb-8.csv', 
           '/Volumes/REU_2023/Per-emb-8_Models/MEGACOMBINED.csv'
          )

In [5]:
#This is for the model remakes we did in July 2025. Just 1 big outfile and I need to add 3 columns for bools for 
#IRE, Keplerian, and Blended

OutputTable = ConvertOutputToDataframe('../../../../Downloads/B5-IRS1_test_bigger_grid_CBmin1.out', 'B5-IRS1', True)
# OutputTable
ConvertDataframeToCSV(OutputTable, 'B5-IRS1_NewBatch.csv')

BMin= 0.294
BMaj= 0.448
Pixsize= 0.054
BeamAreaPix= 141.9017323695538
../../../../Downloads/B5-IRS1_test_bigger_grid_CBmin1.out sucessfully read
5430.0  models were compared
File sucessfully extracted
File sucessfully converted to dataframe


In [18]:
glob.glob('../../../../Downloads/B5-IRS1_test_bigger_grid*')

['../../../../Downloads\\B5-IRS1_test_bigger_grid.out']